# Análisis Completo: Predicción de No-Show Médico

**Proyecto I - Especialización en Ciencia de Datos e IA**

## Objetivo
Predecir la probabilidad de que un paciente no asista a su cita médica, permitiendo a clínicas y hospitales optimizar sus recursos y reducir costos.

## 1. Importación de Librerías

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from src.data_loader import prepare_data
from src.preprocessing import (
    create_preprocessing_pipeline,
    prepare_features_target,
    get_feature_names
)
from src.synthetic_data import generate_synthetic_data, get_synthetic_data_stats
from src.models import (
    train_random_forest,
    train_logistic_regression,
    evaluate_model,
    get_feature_importance,
    compare_models,
    save_model
)
from src.visualization import (
    plot_target_distribution,
    plot_age_distribution,
    plot_noshow_by_feature,
    plot_correlation_matrix,
    plot_feature_importance,
    plot_model_comparison,
    plot_chronic_conditions
)

import warnings
warnings.filterwarnings('ignore')

## 2. Carga y Exploración de Datos

In [ ]:
DATA_PATH = '../data/KaggleV2-May-2016.csv'

df, df_original = prepare_data(DATA_PATH)

df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

## 3. Análisis Exploratorio de Datos (EDA)

### 3.1 Distribución de la Variable Objetivo

In [ ]:
fig = plot_target_distribution(df)
fig.show()

### 3.2 Distribución de Edades

In [ ]:
fig = plot_age_distribution(df)
fig.show()

### 3.3 Condiciones Crónicas

In [ ]:
fig = plot_chronic_conditions(df)
fig.show()

### 3.4 Asistencia por Recepción de SMS

In [ ]:
fig = plot_noshow_by_feature(df, 'SMS_received')
fig.show()

### 3.5 Matriz de Correlación

In [ ]:
numeric_cols = ['Age', 'DaysAdvance', 'AppointmentWeekday', 'AppointmentMonth', 
                'ChronicConditionsCount', 'No-show']

fig = plot_correlation_matrix(df, numeric_cols)
fig.show()

## 4. Preparación de Datos

In [ ]:
X, y = prepare_features_target(df)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

f"Datos de entrenamiento: {X_train.shape[0]}, Datos de prueba: {X_test.shape[0]}"

## 5. Pipeline de Preprocesamiento

In [ ]:
preprocessor = create_preprocessing_pipeline()

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

f"Dimensión después del preprocesamiento: {X_train_processed.shape}"

## 6. Generación de Datos Sintéticos (SMOTE)

In [ ]:
X_train_synthetic, y_train_synthetic = generate_synthetic_data(
    X_train_processed, y_train
)

stats = get_synthetic_data_stats(
    X_train_processed, y_train,
    X_train_synthetic, y_train_synthetic
)

pd.DataFrame([stats]).T

## 7. Entrenamiento de Modelos

### 7.1 Modelos con Datos Originales

In [ ]:
rf_original = train_random_forest(X_train_processed, y_train)
lr_original = train_logistic_regression(X_train_processed, y_train)

"Modelos entrenados con datos originales"

### 7.2 Modelos con Datos Sintéticos

In [ ]:
rf_synthetic = train_random_forest(X_train_synthetic, y_train_synthetic)
lr_synthetic = train_logistic_regression(X_train_synthetic, y_train_synthetic)

"Modelos entrenados con datos sintéticos"

## 8. Evaluación de Modelos

In [ ]:
results = {
    'RF_Original': evaluate_model(rf_original, X_test_processed, y_test),
    'LR_Original': evaluate_model(lr_original, X_test_processed, y_test),
    'RF_Synthetic': evaluate_model(rf_synthetic, X_test_processed, y_test),
    'LR_Synthetic': evaluate_model(lr_synthetic, X_test_processed, y_test)
}

comparison_df = compare_models(results)
comparison_df

In [ ]:
fig = plot_model_comparison(comparison_df)
fig.show()

## 9. Importancia de Variables

In [ ]:
feature_names = get_feature_names(preprocessor, X_train)

importance_df = get_feature_importance(rf_synthetic, feature_names, top_n=10)
importance_df

In [ ]:
fig = plot_feature_importance(importance_df)
fig.show()

## 10. Guardar Modelos

In [ ]:
save_model(rf_synthetic, '../models/random_forest_model.pkl')
save_model(lr_synthetic, '../models/logistic_regression_model.pkl')
save_model(preprocessor, '../models/preprocessor.pkl')

"Modelos guardados exitosamente"

## 11. Conclusiones

### Hallazgos Principales:

1. **Desbalance de Clases**: La clase minoritaria (no-show) representa aproximadamente 20% de los datos.

2. **Impacto de Datos Sintéticos**: La generación de datos sintéticos mediante SMOTE mejoró el desempeño de los modelos, especialmente en métricas como recall y F1-score.

3. **Comparación de Modelos**: 
   - Random Forest mostró mejor desempeño general
   - La Regresión Logística es más interpretable pero menos precisa

4. **Variables Relevantes**: Las variables más importantes para predecir no-show son:
   - Días de anticipación (DaysAdvance)
   - Edad del paciente
   - Recepción de SMS recordatorio

### Aplicaciones:
- Sistema de alertas tempranas para citas con alto riesgo de no-show
- Optimización de envío de recordatorios SMS
- Mejor gestión de agendas médicas